In [9]:
import gensim.models
from gensim import utils
import pandas as pd
import numpy as np
import time

# read the fp results
# too small corpus
fp_tab = pd.read_csv("../../output/debug/false_positive_second.csv")
# feed entire tables
tab_A = pd.read_csv("../../output/buffer/clean_A.csv")
tab_B = pd.read_csv("../../output/buffer/clean_B.csv")

l_title = tab_A["title"].tolist()
r_title = tab_B["title"].tolist()
fp_l_title = fp_tab["ltable_title"].tolist()
fp_r_title = fp_tab["rtable_title"].tolist()

# get corpus
raw_text = l_title + r_title
    
# tokenize
corpus = []
for i, line in enumerate(raw_text):
    toks = utils.simple_preprocess(line)
    corpu = gensim.models.doc2vec.TaggedDocument(toks, [i])
    corpus.append(corpu)
    
# train
st = time.time()
doc2vec = gensim.models.doc2vec.Doc2Vec(vector_size=128, min_count=2, epochs=40)
doc2vec.build_vocab(corpus)
doc2vec.train(corpus, total_examples=doc2vec.corpus_count, epochs=doc2vec.epochs)
print(f"corpus size : {len(corpus)}, training time : {time.time() - st}")

# prepare vectors
pro_l_title = [utils.simple_preprocess(doc) for doc in fp_l_title]
pro_r_title = [utils.simple_preprocess(doc) for doc in fp_r_title]
vec_l_title = [doc2vec.infer_vector(doc) for doc in pro_l_title]
vec_r_title = [doc2vec.infer_vector(doc) for doc in pro_r_title]
    
# new cosine
new_cos_val = [np.dot(l_vec, r_vec) / (np.linalg.norm(l_vec) * np.linalg.norm(r_vec)) \
               for _, (l_vec, r_vec) in enumerate(zip(vec_l_title, vec_r_title))]

# save to disk
fp_tab["new_cosine"] = new_cos_val
fp_tab["diff_cosine"] = fp_tab["cosine"] - fp_tab["new_cosine"]
fp_tab.to_csv("../../output/debug/false_positive_second.csv", index=False)

corpus size : 4589, training time : 10.042770385742188
